# Linear deformation of elastic material with two circular inclusions with prescribed tractions

This example is implemented in a Python file <a href="elasticity_tractions.py" target="_blank">elasticity_tractions.</a> and it illustrates how to:

- Use subdamins
- Use Lagrange multipliers
- Use `UserExpression` class
- Extract normal vector to the physical boundary

## Equation and problem definition

We consider linear deformation of a 2D elastic material ($\Omega_1$) with two circular inclusions ($\Omega_2$, $\Omega_3$) by prescribing tractions on the left ($\Gamma_1$) and right ($\Gamma_2$) boundaries, while the other other boundaries are traction-free ($\Gamma_3$, $\Gamma_4$).
<div align="center">    
    <img src="figs/elasticity_tractions_domain.png" style="width: 520px;"/>
</div>

This problem can be solved via minimization of the free energy
$$F[u_i]=E_{el}[u_i] - W[u_i]=\int_\Omega {\bf dx} \ \frac{1}{2} \sigma_{ij} \epsilon_{ij} - \oint_\Gamma ds\, u_i t_i,$$
with respect to displacements $u_i$. The first term describes the stored elastic energy and the second term describes the work of external tractions ($t_i = \sigma_{ij}^0 n_j$ with $n_j$ the unit normal vector to the boundary). The constitutive equations for all materials are 
$$\sigma_{ij} = 2 \mu \epsilon_{ij} + \lambda \epsilon_{kk} \delta_{ij},$$
$$\epsilon_{ij} = \frac{1}{2} \left(\partial_i u_j + \partial_j u_i\right),$$

where $\lambda$ and $\mu$ are 2D Lame constants that can be expressed in terms of the 2D Young's modulus $E$ and the Poisson's ratio $\nu$ as $\mu=E/[2(1+\nu)]$ and $\lambda = E \nu/(1-\nu^2)$. In this example we use values: $E_1=1$, $E_1=1$, $E_2=10$, $E_3=0.1$, $\nu_1=0.3$, $\nu_2=0.2$, and $\nu_1=0.1$. 

Note that the minimization of the total free energy automatically satisfies the boundary conditions at different materials, which are the continuity of displacements and tractions. In order to prevent rigid body motions (2 translations, 1 rotation), which do not cost any energy, we use Lagrange multipliers ${\bf c}_{trans}$ and $c_{rot}$ and minimize the total free energy 
$$F[u_i]=\int_\Omega {\bf dx} \, \frac{1}{2} \sigma_{ij} \epsilon_{ij} - \oint_\Gamma ds\, u_i t_i +\int_\Omega {\bf dx}\,  c_{trans,i} u_i + \int_\Omega {\bf dx}\  c_{rot} (x u_y -y u_x)$$

## Implementation

Import relevant libraries

In [1]:
from __future__ import print_function
from fenics import *
from dolfin import *
from mshr import *
import matplotlib.pyplot as plt

Create a rectangular box with two circular subdomains. For each subdomain we can prescribe the identifier marker numbers via the `set_subdomain` function. Here we first specified the marker value 1 for the whole rectangular domain and then we specified marker values 2 and 3 for the two circular domains. Note that if subdomains overlap, then the last specified marker is used. Thus the marker value 1 only describes the rectangular region without the two circular disks. Note also that the marker values of 0 cannot be used, because they are reserved to specify the whole domain. To access the values of markers for each cell in the mesh, we use the `MeshFunction`, where `d` is the dimensionality of the cell. 

In [2]:
# Create rectangular mesh with two circular inclusions
N=100
L=1
R_2=0.05
R_3=0.08
domain = Rectangle(Point(-L/2,-L/2),Point(L/2,L/2))
# mark subdomains with markers 1, 2, 3
domain.set_subdomain(1, Rectangle(Point(-L/2,-L/2),Point(L/2,L/2)))
domain.set_subdomain(2, Circle(Point(0.,0.43), R_2))
domain.set_subdomain(3, Circle(Point(-0.15,0.35), R_3))
mesh = generate_mesh(domain, N)
d = mesh.topology().dim() # dimensionality of the problem
print("d = ",d)
markers = MeshFunction("size_t", mesh, d , mesh.domains())

d =  2


To define boundary subdomains we use the `SubDomain` class. Note that we use the `near` function to test whether the mesh points belong to the boundary. This is because we cannot check whether the two real numbers are equal as `x[0]==-L/2`, because of the rounding errors. To mark the values of markers for facets on each boundary subdomain, we again use the `MeshFunction` class, which now has dimensionality `d-1`. Note again that the marker values of 0 cannot be used, because they are reserved to specify the whole boundary domain.

In [3]:
# define boundary subdomains
class Left(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[0], -L/2)

class Right(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[0], +L/2)

class Top(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[1], L/2)

class Bottom(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[1], -L/2)

left = Left()
right = Right()
top = Top()
bottom = Bottom()

# mark boundary subdomains with markers 1, 2, 3, 4
boundaries = MeshFunction("size_t", mesh, d-1, 0)
boundaries.set_all(0)
left.mark(boundaries, 1)
right.mark(boundaries, 2)
top.mark(boundaries, 3)
bottom.mark(boundaries, 4)

The `UserExpression` class can be used to specify elastic constants on each of the three subdomains $\Omega_1$, $\Omega_2$, $\Omega_3$ with the help of cell markers that were defined above

In [4]:
# elastic constants of the matrix and two circular inclusions
E_1=1
E_2=10
E_3=0.1
nu_1=0.3
nu_2=0.2
nu_3=0.1

# define class for calculating the Young's modulus over the whole domain
class E_class(UserExpression):
    def __init__(self, **kwargs):
        self.markers = markers
        super().__init__(**kwargs)
    def eval_cell(self, value, x, ufc_cell):
        if markers[ufc_cell.index] == 1:
            value[0] = E_1
        elif markers[ufc_cell.index] == 2:
            value[0] = E_2
        else:
            value[0] = E_3

# define class for calculating the Poisson's ratio over the whole domain
class nu_class(UserExpression):
    def __init__(self, **kwargs):
        self.markers = markers
        super().__init__(**kwargs)
    def eval_cell(self, value, x, ufc_cell):
        if markers[ufc_cell.index] == 1:
            value[0] = nu_1
        elif markers[ufc_cell.index] == 2:
            value[0] = nu_2
        else:
            value[0] = nu_3

# functions of elastic constants on the whole domain
E = E_class(degree=1)
nu = nu_class(degree=1)
mu=E/2/(1+nu)
Lambda=E*nu/(1-nu*nu)

Use `MixedElement` to specify the function space for displacement vectors (linear Lagrange elements) and for 3 Lagrange multipliers (real numbers). 

In [17]:
#define function space with mixed finite elements (displacements + 3 Lagrange multipliers)
degreeElements = 1
P1 = FiniteElement('Lagrange', mesh.ufl_cell(), degreeElements)
R = FiniteElement('Real', mesh.ufl_cell(), 0)
MFS = FunctionSpace(mesh, MixedElement([(P1*P1),(R*R),R]))

The function on this `FunctionSpace` can be defined as usual. However, it is convenient to `split` these function into functions that correspond to subspaces for displacemetns $u$ and Lagrange multipliers ${\bf c_{trans}}$ and $c_{rot}$.

In [18]:
#define function and split it into displacements u and Lagrange multipliers
f  = Function(MFS)
u, c_trans, c_rot = split(f)
#define test function
tf = TestFunction(MFS)

The strain and stress tensor are defined as discussed in the elasticity example with clamped-free boundary conditions (<a href="elasticity_clamped.ipynb" target="_blank">Jupyter notebook</a>, <a href="elasticity_clamped.html" target="_blank">HTML</a>).

In [8]:
# define strain and stress
def epsilon(u):
    return sym(grad(u))
def sigma(u):
    return 2*mu*epsilon(u) + Lambda*tr(epsilon(u))*Identity(d)

To prescribe tractions at the boundary $t_i=\sigma_{ij}^0 n_j$, we use the `FacetNormal` function to obtain the unit normal vector $n_j$ to the boundary.

In [23]:
#external load
sigma_xx = 0.2*E_1
sigma_xy = 0
sigma_yy = 0
sigma_0 = Constant(((sigma_xx,sigma_xy),(sigma_xy,sigma_yy)))

#unit normal vector to the boundary
n = FacetNormal(mesh)
                   
#tractions on boundaries
t = sigma_0 * n

The toal free energy can be decomposed to the elastic energy
$$E_{el}[u_i]=\int_\Omega {\bf dx} \, \frac{1}{2} \sigma_{ij} \epsilon_{ij},$$
work of external forces
$$W[u_i]=\oint_\Gamma ds\, u_i t_i$$
and constraints with Lagrange multipliers
$$\int_\Omega {\bf dx}\,  c_{trans,i} u_i + \int_\Omega {\bf dx}\  c_{rot} (x u_y -y u_x)$$

In [24]:
#calculate elastic energy
elastic_energy = 1/2*inner(sigma(u),epsilon(u))*dx
#calculate work of external tractions
work = dot(t,u)*ds
#Lagrange multipliers to prevent rigid body motions
r=Expression(('x[0]','x[1]'),degree=1)
constraints = dot(c_trans,u)*dx + c_rot*(r[0]*u[1]-r[1]*u[0])*dx
#total free energy
free_energy =  elastic_energy - work + constraints

<div class="alert alert-block alert-info">
Note that if the 3 materials had different constitutive laws we coud have evaluated the total elastic energy by integrating separately over domains $\Omega_1$, $\Omega_2$, and $\Omega_3$.<br> 
<code>dx = Measure('dx', domain=mesh, subdomain_data=markers)
elastic_energy = energy_density1*dx(1) + energy_density2*dx(2) + energy_density3*dx(3) 
</code>
Similarly, if we had different values of tractions on different boundary domains we could have evaluated the work by integrating over different domains.
<br> 
<code>ds = Measure('ds', domain=mesh, subdomain_data=boundaries)
work = dot(t1,u)*ds(1) + dot(t2,u)*ds(2) + dot(t3,u)*ds(3)
</code>
</div>

Minimize the total free energy and calculate the contributions from the elastic energy, work of external loads and from constraints

In [21]:
#minimize total free energy
Res = derivative(free_energy, f, tf)
solve(Res == 0, f)

#calculate total free energy
print("Tot Free Energy = ",assemble(free_energy))
print("Elastic Energy = ",assemble(elastic_energy))
print("Work = ",assemble(work))
print("Constraints = ",assemble(constraints))

Tot Free Energy =  -0.020811012183109497
Elastic Energy =  0.02081101218310997
Work =  0.0416220243662194
Constraints =  -2.1385721704735844e-31


We can export displacements and stress for the visualization in <a href="https://www.paraview.org/" target="_blank">ParaView</a> as was done in the elasticity example with clamped-free boundary conditions (<a href="elasticity_clamped.ipynb" target="_blank">Jupyter notebook</a>, <a href="elasticity_clamped.html" target="_blank">HTML</a>).

In [27]:
# export displacements
VFS = VectorFunctionSpace(mesh, 'Lagrange', 1)
disp=project(u, VFS)
disp.rename("displacements","")
fileD = File("data/tractions_displacement.pvd");
fileD << disp;

# calculate and export von Mises stress
FS = FunctionSpace(mesh, 'Lagrange', 1)
devStress = sigma(u) - (1./d)*tr(sigma(u))*Identity(d)  # deviatoric stress
von_Mises = project(sqrt(3./2*inner(devStress, devStress)), FS)
von_Mises.rename("von Mises","")
fileS = File("data/tractions_vonMises_stress.pvd");
fileS << von_Mises;

# calculate and export stress component sigma_xx
sigma_xx = project(sigma(u)[0,0], FS)
sigma_xx.rename("sigma_xx","")
fileS = File("data/tractions_sigma_xx.pvd");
fileS << sigma_xx;

# calculate and export stress component sigma_yy
sigma_yy = project(sigma(u)[1,1], FS)
sigma_yy.rename("sigma_yy","")
fileS = File("data/tractions_sigma_yy.pvd");
fileS << sigma_yy;

# calculate and export stress component sigma_xy
sigma_xy = project(sigma(u)[0,1], FS)
sigma_xy.rename("sigma_xy","")
fileS = File("data/tractions_sigma_xy.pvd");
fileS << sigma_xy;

# export Young's modulus
young = project(E, FS)
young.rename("Young's modulus","")
fileS = File("data/tractions_young.pvd");
fileS << young;


## Complete code

In [29]:
from __future__ import print_function
from fenics import *
from dolfin import *
from mshr import *
import matplotlib.pyplot as plt

# Create rectangular mesh with two circular inclusions
N=100
L=1
R_2=0.05
R_3=0.08
domain = Rectangle(Point(-L/2,-L/2),Point(L/2,L/2))
# mark subdomains with markers 1, 2, 3
domain.set_subdomain(1, Rectangle(Point(-L/2,-L/2),Point(L/2,L/2)))
domain.set_subdomain(2, Circle(Point(0.,0.43), R_2))
domain.set_subdomain(3, Circle(Point(-0.15,0.35), R_3))
mesh = generate_mesh(domain, N)
d = mesh.topology().dim() # dimensionality of the problem
markers = MeshFunction("size_t", mesh, d , mesh.domains())


# define boundary subdomains
class Left(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[0], -L/2)

class Right(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[0], +L/2)

class Top(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[1], L/2)

class Bottom(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[1], -L/2)

left = Left()
right = Right()
top = Top()
bottom = Bottom()

# mark boundary subdomains with markers 1, 2, 3, 4
boundaries = MeshFunction("size_t", mesh, d-1, 0)
boundaries.set_all(0)
left.mark(boundaries, 1)
right.mark(boundaries, 2)
top.mark(boundaries, 3)
bottom.mark(boundaries, 4)


# elastic constants of the matrix and two circular inclusions
E_1=1
E_2=10
E_3=0.1
nu_1=0.3
nu_2=0.2
nu_3=0.1



# define class for calculating the Young's modulus over the whole domain
class E_class(UserExpression):
    def __init__(self, **kwargs):
        self.markers = markers
        super().__init__(**kwargs)
    def eval_cell(self, value, x, ufc_cell):
        if markers[ufc_cell.index] == 1:
            value[0] = E_1
        elif markers[ufc_cell.index] == 2:
            value[0] = E_2
        else:
            value[0] = E_3

# define class for calculating the Poisson's ratio over the whole domain
class nu_class(UserExpression):
    def __init__(self, **kwargs):
        self.markers = markers
        super().__init__(**kwargs)
    def eval_cell(self, value, x, ufc_cell):
        if markers[ufc_cell.index] == 1:
            value[0] = nu_1
        elif markers[ufc_cell.index] == 2:
            value[0] = nu_2
        else:
            value[0] = nu_3

# functions of elastic constants on the whole domain
E = E_class(degree=1)
nu = nu_class(degree=1)
mu=E/2/(1+nu)
Lambda=E*nu/(1-nu*nu)




#define function space with mixed finite elements (displacements + 3 Lagrange multipliers)
degreeElements = 1
P1 = FiniteElement('Lagrange', mesh.ufl_cell(), degreeElements)
R = FiniteElement('Real', mesh.ufl_cell(), 0)
MFS = FunctionSpace(mesh, MixedElement([(P1*P1),(R*R),R]))

#define function and split it into displacements u and Lagrange multipliers
f  = Function(MFS)
u, c_trans, c_rot = split(f)
#define test function
tf = TestFunction(MFS)


# define strain and stress
def epsilon(u):
    return sym(grad(u))
def sigma(u):
    return 2*mu*epsilon(u) + Lambda*tr(epsilon(u))*Identity(d)

#external load
sigma_xx = 0.2*E_1
sigma_xy = 0
sigma_yy = 0
sigma_0 = Constant(((sigma_xx,sigma_xy),(sigma_xy,sigma_yy)))

#unit normal vector to the boundary
n = FacetNormal(mesh)
                   
#tractions on boundaries
t = sigma_0 * n

#calculate elastic energy
elastic_energy = 1/2*inner(sigma(u),epsilon(u))*dx
#calculate work of external tractions
work = dot(t,u)*ds
#Lagrange multipliers to prevent rigid body motions
r=Expression(('x[0]','x[1]'),degree=1)
constraints = dot(c_trans,u)*dx + c_rot*(r[0]*u[1]-r[1]*u[0])*dx
#total free energy
free_energy =  elastic_energy - work + constraints


#minimize total free energy
Res = derivative(free_energy, f, tf)
solve(Res == 0, f)

#calculate total free energy
print("Tot Free Energy = ",assemble(free_energy))
print("Elastic Energy = ",assemble(elastic_energy))
print("Work = ",assemble(work))
print("Constraints = ",assemble(constraints))


# export displacements
VFS = VectorFunctionSpace(mesh, 'Lagrange', 1)
disp=project(u, VFS)
disp.rename("displacements","")
fileD = File("data/tractions_displacement.pvd");
fileD << disp;

# calculate and export von Mises stress
FS = FunctionSpace(mesh, 'Lagrange', 1)
devStress = sigma(u) - (1./d)*tr(sigma(u))*Identity(d)  # deviatoric stress
von_Mises = project(sqrt(3./2*inner(devStress, devStress)), FS)
von_Mises.rename("von Mises","")
fileS = File("data/tractions_vonMises_stress.pvd");
fileS << von_Mises;

# calculate and export stress component sigma_xx
sigma_xx = project(sigma(u)[0,0], FS)
sigma_xx.rename("sigma_xx","")
fileS = File("data/tractions_sigma_xx.pvd");
fileS << sigma_xx;

# calculate and export stress component sigma_yy
sigma_yy = project(sigma(u)[1,1], FS)
sigma_yy.rename("sigma_yy","")
fileS = File("data/tractions_sigma_yy.pvd");
fileS << sigma_yy;

# calculate and export stress component sigma_xy
sigma_xy = project(sigma(u)[0,1], FS)
sigma_xy.rename("sigma_xy","")
fileS = File("data/tractions_sigma_xy.pvd");
fileS << sigma_xy;

# export Young's modulus
young = project(E, FS)
young.rename("Young's modulus","")
fileS = File("data/tractions_young.pvd");
fileS << young;

Tot Free Energy =  -0.020811012183109497
Elastic Energy =  0.02081101218310997
Work =  0.0416220243662194
Constraints =  -2.1385721704735844e-31
